In [ ]:
!pip install kaggle
! mkdir ~/.kaggle
print("\nOpen Kaggle Json API file for dataset download: ")
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c digit-recognizer

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from zipfile import ZipFile
import shutil

CHUNK_SIZE = 40960
KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)
destination_path = os.path.join(KAGGLE_INPUT_PATH, "")

with ZipFile('digit-recognizer.zip') as zfile:
  zfile.extractall(destination_path)
print('Data source import complete.')

In [ ]:
import numpy as np
import pandas as pd
import torch

In [ ]:
np.random.seed(2)
torch.manual_seed(2)

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageOps, ImageEnhance
import math
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class dataset(Dataset):
    def __init__(self, file_path, transform=transforms.Compose([transforms.ToPILImage(),
                                                                transforms.ToTensor(),
                                                                transforms.Normalize(mean=(0.5,),
                                                                                     std=(0.5,))])):
        df = pd.read_csv(file_path)
        if len(df.columns)==n_pixels:
            self.X = df.values.reshape((-1, 28, 28)).astype(np.uint8)[:, :, :, None]
            self.y = None
        else:
            self.X = df.iloc[:, 1:].values.reshape((-1, 28, 28)).astype(np.uint8)[:, :, :, None]
            self.y = torch.from_numpy(df.iloc[:, 0].values)
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.transform(self.X[idx]), self.y[idx]
        return self.transform(self.X[idx])

In [ ]:
test_df = pd.read_csv('../input/test.csv')
test_df.head()

In [ ]:
n_pixels = len(test_df.columns)
n_pixels

In [ ]:
num_workers = 0
batch_size = 64
transform = transforms.Compose([transforms.ToPILImage(), transforms.RandomRotation(degrees=20),
                                transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

train_dataset = dataset('../input/train.csv', transform=transform)
test_dataset = dataset('../input/test.csv')
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
dataiter = iter(train_dl)
images, labels = next(dataiter)
images = images.numpy()

fig = plt.figure(figsize=(25, 4))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, int(batch_size/2), idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    ax.set_title(str(labels[idx].item()))

In [ ]:
import torch.nn as nn

In [ ]:
class ConvLayer(nn.Module):
    def __init__(self, in_channels=1, out_channels=256):
        super(ConvLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=9, stride=1, padding=0)
    def forward(self, x):
        x = F.relu(self.conv(x))
        return x

In [ ]:
class PrimaryCaps(nn.Module):
    def __init__(self, num_capsules=8, in_channels=256, out_channels=32):
        super(PrimaryCaps, self).__init__()
        self.capsules = nn.ModuleList([
            nn.Conv2d(in_channels, out_channels, kernel_size=9, stride=2, padding=0)
            for _ in range(num_capsules)
        ])
    def forward(self, x):
        batch_size = x.size(0)
        u = [capsule(x).view(batch_size, 32*6*6, 1) for capsule in self.capsules]
        u = torch.cat(u, dim=-1)
        u_squashed = self.squash(u)
        return u_squashed

    def squash(self, x):
        squared_norm = (x**2).sum(dim=-1, keepdim=True)
        scale = squared_norm/(1+squared_norm)
        output = scale * x/torch.sqrt(squared_norm)
        return output

In [ ]:
def softmax(x, dim=1):
    transposed_inp = x.transpose(dim, len(x.size())-1)
    softmaxed = F.softmax(transposed_inp.contiguous().view(-1, transposed_inp.size(-1)), dim=-1)
    return softmaxed.view(*transposed_inp.size()).transpose(dim, len(x.size())-1)

In [ ]:
def dynamic_routing(b_ij, u_hat, squash, routing_iterations=3):
    for iterations in range(routing_iterations):
        c_ij = softmax(b_ij, dim=2)
        s_j = (c_ij*u_hat).sum(dim=2, keepdim=True)
        v_j = squash(s_j)
        if iterations < routing_iterations-1:
            a_ij = (u_hat * v_j).sum(dim=-1, keepdim=True)
            b_ij = b_ij + a_ij
    return v_j

In [ ]:
TRAIN_ON_GPU = torch.cuda.is_available()
if TRAIN_ON_GPU: print('training on gpu')

In [ ]:
class DigitCaps(nn.Module):
    def __init__(self, num_caps=10, previous_layer_nodes=32*6*6,
                 in_channels=8, out_channels=16):
        super(DigitCaps, self).__init__()
        self.num_caps = num_caps
        self.previous_layer_nodes = previous_layer_nodes
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.W = nn.Parameter(torch.randn(num_caps, previous_layer_nodes,
                                          in_channels, out_channels))

    def forward(self, x):
        x = x[None, :, :, None, :]
        W = self.W[:, None, :, :, :]
        x_hat = torch.matmul(x, W)
        b_ij = torch.zeros(*x_hat.size())
        if TRAIN_ON_GPU: b_ij = b_ij.cuda()
        v_j = dynamic_routing(b_ij, x_hat, self.squash, routing_iterations=3)
        return v_j

    def squash(self, x):
        squared_norm = (x**2).sum(dim=-1, keepdim=True)
        scale = squared_norm/(1+squared_norm)
        out = scale * x/torch.sqrt(squared_norm)
        return out

In [ ]:
class Decoder(nn.Module):
    def __init__(self, input_vector_length=16, input_capsules=10, hidden_dim=512):
        super(Decoder, self).__init__()
        input_dim = input_vector_length*input_capsules
        self.lin_layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, hidden_dim*2),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim*2, 28*28),
            nn.Sigmoid()
        )
    def forward(self, x):
        classes = (x**2).sum(dim=-1)**0.5
        classes = F.softmax(classes, dim=-1)
        _, max_length_indices = classes.max(dim=1)
        sparse_matrix = torch.eye(10)
        if TRAIN_ON_GPU: sparse_matrix = sparse_matrix.cuda()
        y = sparse_matrix.index_select(dim=0, index=max_length_indices.data)
        x = x*y[:, :, None]
        # flattened_x = x.view(x.size(0), -1)
        flattened_x = x.reshape(x.size(0), -1)
        reconstructed = self.lin_layers(flattened_x)
        return reconstructed, y

In [ ]:
class CapsuleNetwork(nn.Module):
    def __init__(self):
        super(CapsuleNetwork, self).__init__()
        self.conv_layer = ConvLayer()
        self.primary_capsule = PrimaryCaps()
        self.digit_capsule = DigitCaps()
        self.decoder = Decoder()
    def forward(self, x):
        primary_caps_out = self.primary_capsule(self.conv_layer(x))
        caps_out = self.digit_capsule(primary_caps_out).squeeze().transpose(0, 1)
        reconstructed, y = self.decoder(caps_out)
        return caps_out, reconstructed, y

In [ ]:
capsule_net = CapsuleNetwork()
print(capsule_net)
if TRAIN_ON_GPU: capsule_net = capsule_net.cuda()

In [ ]:
class CapsuleLoss(nn.Module):
    def __init__(self):
        super(CapsuleLoss, self).__init__()
        self.reconstruction_loss = nn.MSELoss(size_average=False)

    def forward(self, x, labels, images, reconstructions):
        batch_size = x.size(0)
        v_c = torch.sqrt((x**2).sum(dim=2, keepdim=True))
        left = F.relu(0.9-v_c).view(batch_size, -1)
        right = F.relu(v_c-0.1).view(batch_size, -1)
        margin_loss = labels * left + 0.5 * (1.-labels) * right
        margin_loss = margin_loss.sum()
        images = images.view(reconstructions.size()[0], -1)
        reconstruction_loss = self.reconstruction_loss(reconstructions, images)
        return (margin_loss + 0.0005 * reconstruction_loss)/images.size(0)

In [ ]:
import torch.optim as optim
criterion = CapsuleLoss()
optimizer = optim.Adam(capsule_net.parameters())

In [ ]:
def train(capsule_net, criterion, optimizer, n_epochs, print_every=300):
    losses = []
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        capsule_net.train()
        for batch_i, (images, target) in enumerate(train_dl):
            target = torch.eye(10).index_select(dim=0, index=target)
            if TRAIN_ON_GPU: images, target = images.cuda(), target.cuda()
            optimizer.zero_grad()
            caps_output, reconstructions, y = capsule_net(images)
            loss = criterion(caps_output, target, images, reconstructions)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            if batch_i != 0 and batch_i % print_every == 0:
                avg_train_loss = train_loss/print_every
                losses.append(avg_train_loss)
                print('Epoch: {} \tTraining Loss: {:.8f}'.format(epoch, avg_train_loss))
                train_loss = 0
    return losses

In [ ]:
n_epochs = 10

In [ ]:
losses = train(capsule_net, criterion, optimizer, n_epochs=n_epochs)

In [ ]:
plt.plot(losses)
plt.title('Training Loss')
plt.show()

In [ ]:
out = []
capsule_net.eval()
for image in test_dl:
    if TRAIN_ON_GPU: image = image.cuda()
    caps_out, reconstructed, y = capsule_net(image)
    _, pred = torch.max(y.data.cpu(), 1)
    out.extend(pred.numpy().tolist())

len(out)
sub = pd.read_csv('../input/sample_submission.csv')
sub['Label'] = out
sub.to_csv('capsule.csv', index=False)